I tried a simple pre-processing

In [ ]:
#this one works best
import os
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

# Initialize stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Regular expression to match words, including splitting hyphenated words
    tokens = regexp_tokenize(text, pattern=r'\s|[\.,;\'\"\-!\*]', gaps=True)

    # Split hyphenated words into separate words
    fixed_tokens = []
    for token in tokens:
        if '-' in token:  # Split tokens with hyphens into separate words
            fixed_tokens.extend(token.split('-'))
        else:
            fixed_tokens.append(token)

    # Lowercasing and removing punctuation and stopwords
    filtered_tokens = [
        word.lower() for word in fixed_tokens
        if word.isalnum() and word.lower() not in stop_words
    ]

    return " ".join(filtered_tokens)

# Assuming your articles are in a folder called "news_articles"
folder_path = "news_articles"
preprocessed_texts = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Read the file and preprocess the text
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        cleaned_text = preprocess_text(text)
        preprocessed_texts.append(cleaned_text)

# Print an example of the preprocessed text (for the first article)
print(preprocessed_texts[1])  # prints the selected processed article for testing


Boolean Model

Due to the coding taking a lot of space not all words show up and it says "The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it."

In [ ]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')

# Preprocess the text (tokenization, stopword removal)
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords and non-alphanumeric words
    return filtered_tokens

# Load all .txt files into a dictionary
def load_documents(directory):
    documents = {}
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                documents[filename] = preprocess(file.read())
    return documents

# Build the Boolean term-document matrix
def build_boolean_model(documents):
    all_terms = set()
    for doc in documents.values():
        all_terms.update(doc)

    term_document_matrix = {}
    for term in all_terms:
        term_document_matrix[term] = {}
        for doc_name, doc_terms in documents.items():
            term_document_matrix[term][doc_name] = 1 if term in doc_terms else 0
    return term_document_matrix

# Example usage
directory = 'news_articles'  # Replace with the directory containing your .txt files
documents = load_documents(directory)
boolean_model = build_boolean_model(documents)

# Print the Boolean Model for a sample term
for term, doc_dict in boolean_model.items():
    print(f"Term: {term}")
    for doc_name, presence in doc_dict.items():
        print(f"  {doc_name}: {presence}")


Another approach for the same Boolean Model (I wanted to just test it)

In [ ]:
import os
import string
import re
from collections import defaultdict

# Function to preprocess the text (tokenization, removing punctuation, lowercase)
def preprocess_text(text):
    # Remove punctuation and lowercase all words
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    tokens = text.split()
    return tokens

# Read the files from the news_articles directory
directory = "news_articles"
docs = []
doc_names = []

# Read all text files and preprocess them
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            processed_tokens = preprocess_text(content)
            docs.append(processed_tokens)
            doc_names.append(filename)

# Create a vocabulary (set of unique terms)
vocab = set()
for doc in docs:
    vocab.update(doc)

# Sort the vocabulary alphabetically
sorted_vocab = sorted(vocab)



# Create the Boolean term-document matrix
boolean_matrix = defaultdict(lambda: defaultdict(int))

# Fill the matrix with 1s and 0s (1 if the term exists in the document, 0 otherwise)
for doc_idx, doc in enumerate(docs):
    for term in doc:
        if term in sorted_vocab:
            boolean_matrix[term][doc_names[doc_idx]] = 1

# Print the Boolean Model
print("\nBoolean Model (Term-Document Matrix):")
for term in sorted_vocab:
    print(f"Term: {term}")
    for doc_name in doc_names:
        print(f"  {doc_name}: {boolean_matrix[term].get(doc_name, 0)}")


Inverted Boolean Model - This way would be more efficient

In [ ]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')

# Basic Preprocess function (tokenize, lowercase, and remove non-alphanumeric)
def preprocess_basic(text):
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    filtered_tokens = [word for word in tokens if word.isalnum()]  # Remove non-alphanumeric words
    return filtered_tokens

# Load all .txt files into a dictionary
def load_documents(directory):
    documents = {}
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                documents[filename] = preprocess_basic(file.read())
    return documents

# Build the Inverted Index (Boolean model)
def build_inverted_index(documents):
    inverted_index = {}

    for doc_name, terms in documents.items():
        for term in terms:
            if term not in inverted_index:
                inverted_index[term] = []
            # Add document to the inverted index list if term is present in that document
            if doc_name not in inverted_index[term]:
                inverted_index[term].append(doc_name)

    return inverted_index

# Display the Inverted Index in a readable format
def display_inverted_index(inverted_index):
    for term, doc_list in inverted_index.items():
        print(f"Term: {term}")
        print(f"  Documents: {', '.join(doc_list)}")

# Example usage
directory = 'news_articles'  # Path to your news articles folder
documents = load_documents(directory)
inverted_index = build_inverted_index(documents)
display_inverted_index(inverted_index)


TF-based Vector Space Model

In [ ]:
import os
import string
import re
from collections import defaultdict

# Function to preprocess the text (tokenization, removing punctuation, lowercase)
def preprocess_text(text):
    # Remove punctuation and lowercase all words
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    tokens = text.split()
    return tokens

# Read the files from the news_articles directory
directory = "news_articles"
docs = []
doc_names = []

# Read all text files and preprocess them
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            processed_tokens = preprocess_text(content)
            docs.append(processed_tokens)
            doc_names.append(filename)

# Create a vocabulary (set of unique terms)
vocab = set()
for doc in docs:
    vocab.update(doc)

# Sort the vocabulary alphabetically
sorted_vocab = sorted(vocab)

# Create the Term Frequency (TF) matrix
tf_matrix = defaultdict(lambda: defaultdict(int))

# Fill the matrix with term frequencies (count of term occurrences)
for doc_idx, doc in enumerate(docs):
    for term in doc:
        if term in sorted_vocab:
            tf_matrix[doc_names[doc_idx]][term] += 1

# Print the Term Frequency (TF) Matrix
print("\nVector Space Model (Term Frequency - TF):")
for doc_name in doc_names:
    print(f"\nDocument: {doc_name}")
    for term in sorted_vocab:
        print(f"  {term}: {tf_matrix[doc_name].get(term, 0)}")


I tried this code with a single file to see the output better

In [ ]:
import os
import string
import re
from collections import defaultdict

# Function to preprocess the text (tokenization, removing punctuation, lowercase)
def preprocess_text(text):
    # Remove punctuation and lowercase all words
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    tokens = text.split()
    return tokens

# Read the files from the news_articles directory (single document in this case)
directory = "news_articles"
docs = []
doc_names = []

# Read and preprocess only the specific file (artc4.txt for example)
single_doc_filename = "artc3.txt"  # Change this to the filename you want to test

with open(os.path.join(directory, single_doc_filename), 'r', encoding='utf-8') as file:
    content = file.read()
    processed_tokens = preprocess_text(content)
    docs.append(processed_tokens)
    doc_names.append(single_doc_filename)

# Create a vocabulary (set of unique terms)
vocab = set()
for doc in docs:
    vocab.update(doc)

# Sort the vocabulary alphabetically
sorted_vocab = sorted(vocab)

# Create the Term Frequency (TF) matrix
tf_matrix = defaultdict(lambda: defaultdict(int))

# Fill the matrix with term frequencies (count of term occurrences)
for doc_idx, doc in enumerate(docs):
    for term in doc:
        if term in sorted_vocab:
            tf_matrix[doc_names[doc_idx]][term] += 1

# Print the Term Frequency (TF) Matrix for the single document
print("\nVector Space Model (Term Frequency - TF) for single document:")
for doc_name in doc_names:
    print(f"\nDocument: {doc_name}")
    for term in sorted_vocab:
        print(f"  {term}: {tf_matrix[doc_name].get(term, 0)}")


TF-IDF (Term Frequency-Inverse Document Frequency) Vector Space Model

In [ ]:
import os
import math
from collections import Counter
import re

# Preprocess the text by tokenizing and cleaning
def preprocess(doc):
    # Remove non-alphanumeric characters (optional, depending on your needs)
    doc = re.sub(r'\W+', ' ', doc.lower())
    return doc.split()

# Compute Term Frequency (TF)
def compute_tf(doc):
    tf = Counter(doc)
    total_terms = len(doc)
    return {word: count / total_terms for word, count in tf.items()}

# Compute Inverse Document Frequency (IDF)
def compute_idf(corpus):
    N = len(corpus)
    idf = {}
    doc_count = {}
    for doc in corpus:
        words_in_doc = set(doc)
        for word in words_in_doc:
            if word not in doc_count:
                doc_count[word] = 0
            doc_count[word] += 1

    for word, count in doc_count.items():
        idf[word] = math.log(N / count)

    return idf

# Compute TF-IDF
def compute_tfidf(corpus):
    tfidf = []
    idf = compute_idf(corpus)
    for doc in corpus:
        tf = compute_tf(doc)
        tfidf_doc = {word: tf[word] * idf[word] for word in tf}
        tfidf.append(tfidf_doc)
    return tfidf

# Load the text files from the 'news_articles' directory
def load_documents(directory):
    docs = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
                doc = f.read()
                docs.append(preprocess(doc))
    return docs

# Example usage:
# Assuming your files are located in the 'news_articles' directory
docs = load_documents('news_articles')
tfidf = compute_tfidf(docs)

# Print the TF-IDF results for each document
for idx, doc_tfidf in enumerate(tfidf):
    print(f"Article {idx + 1} :")
    print(doc_tfidf)
    print("\n")


I tried it for one text to see the output better:

In [ ]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the file path
file_path = 'news_articles/artc1.txt'

# Check if the file exists and read the content
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Apply vectorization to the text
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([text])  # Only the single document

    # Get the feature names (words)
    feature_names = vectorizer.get_feature_names_out()

    # Create a dictionary of words and their corresponding TF-IDF scores
    tfidf_dict = {word: float(score) for word, score in zip(feature_names, X.toarray()[0])}

    # Output the dictionary
    print(tfidf_dict)
else:
    print(f"File '{file_path}' not found.")
